In [ ]:
# scrap PDF loop data from toby christie's site
# https://tobychristie.com/loop-data-2021-nascar-cup-series/"
# this is v2 of the scraper because there is a different pattern of PDF that tabula could output
# this works for most of the PDFs from the 2021 and 2022 cup series data

# skip to the last cell and execute, 
# if you get an error then go through each cell to see what is going on
# likely issues with the merged columns.

# tabula is the library that reads in searchable PDFs
# documentation: https://tabula-py.readthedocs.io/en/latest/getting_started.html

# raw data can be found here: /data/lap_data

import tabula
import pandas as pd
from tabula import read_pdf
HEADERS=['Car','Driver','AvgStart','AvgMidRace','AvgFinish','Avg Pos','LapsLed','PercentLapsLed','TotalLaps','DriverRating','Pts','PassDiff','GreenPass','GreenPassed','QualityPasses','PercentQualityPasses','NumFastestLaps','LapsinTop15','PercentLapsTop15']
data_location=r".\SIADS591_592_Milestone1\Milestone_I\Milestone_I\data\loop_TobyChristie\pdf\2022\LD12211_POST_POSTBOOK.pdf"
df=read_pdf(data_location,pages='1')


In [ ]:
# preview uncleaned data
# df[0].head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Box Score,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dover Motor Speedway,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DuraMAX Drydene 400 presented by RelaDyne,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Provided by NASCAR Statistics at 5/2/2022 4:39...,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Green Flag Passes For Lead:2
4,Car,Driver,Start,Mid,Closer,Finish,High,Low Avg Pass Green Green Quality % Quality # F...,Laps in % Laps in,Laps,% Laps Total Driver Pts
5,NaN,NaN,Pos,Race,Pos,Pos,Pos,Pos Pos Diff Pass Passed Passes Passes Laps,Top 15 Top 15,Led,Led Laps Rating
6,9,Chase Elliott,4,8,1,1,1,20 4.1 11 33 22 32 96.97 33,399 99.8,73,18.3 400 132.8 50
7,47,Ricky Stenhouse Jr,15,16,2,2,2,19 8.9 21 59 38 52 88.14 8,376 94.0,0,0.0 400 102.8 41
8,1,Ross Chastain,7,10,3,3,1,20 4.7 14 43 29 42 97.67 38,397 99.3,86,21.5 400 125.2 46
9,20,Christopher Bell,17,30,5,4,2,32 15.3 -2 48 50 18 37.5 13,206 51.5,0,0.0 400 91.5 41


In [ ]:
# dataframe cleanup steps:
# 1. removed extra header information
df2=df[0].iloc[5:]
# 2. handling merged columns 
df2['Box Score']=df2['Box Score'].str.split()
df2['Unnamed: 7']=df2['Unnamed: 7'].str.split()
df2['Unnamed: 9']=df2['Unnamed: 9'].str.split()

<ipython-input-4-c4d1a71ad317>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Box Score']=df2['Box Score'].str.split()
<ipython-input-4-c4d1a71ad317>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Unnamed: 7']=df2['Unnamed: 7'].str.split()
<ipython-input-4-c4d1a71ad317>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [ ]:
# df2.head()

In [ ]:
#split merged columns into the respected columns, this information can be gathered from the PDF.
df3=pd.DataFrame(df2['Box Score'].tolist(), columns=['PassDiff','GreenPass','GreenPassed','QualityPasses','PercentQualityPasses','NumFastestLaps'])
df4=pd.DataFrame(df2['Unnamed: 6'].tolist(), columns=['LapsinTop15','PercentLapsTop15'])
# resetting the index to prep for the concat
df2=df2[['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11']]
df2=df2.reset_index(drop=True)
# combine the split up columns with the properly parsed columns
df_final=pd.concat([df2,df3,df4],axis=1)
df_final.columns=HEADERS

In [ ]:
# double checking
# df_final.head(2)

In [ ]:
# final output
df_final.to_csv(r'.\SIADS591_592_Milestone1\Milestone_I\data\loop_TobyChristie\csv\2021\{}.csv'.format(fname_year+fname_month+fname_day+'_'+fname_track),index=False)


In [ ]:
# put it all together and loopiong through a directory where the downloaded PDFs are.
import tabula
import pandas as pd
import re
from tabula import read_pdf
HEADERS_1=["Car","Driver","StartPos","MidRace","CloserPos","FinishPos","HighPos","LowPos", "NumFastestLaps", "LapsLed", "AvgPos","PassDiff","GreenPass","GreenPassed","QualityPasses","PercentQualityPasses", "LapsTop15","PercentLapsTop15", "PercentLapsLed", "TotalLaps","DriverRating","Pts"]
HEADERS_2=["Car","Driver","StartPos","MidRace","CloserPos","FinishPos","HighPos","LowPos","AvgPos","PassDiff","GreenPass","GreenPassed","QualityPasses","PercentQualityPasses","NumFastestLaps","LapsTop15","PercentLapsTop15","LapsLed","PercentLapsLed","TotalLaps","DriverRating","Pts"]


# loop through the whole directory
directory=r'.\SIADS591_592_Milestone1\Milestone_I\Milestone_I\data\loop_TobyChristie\pdf\2022'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        # read in data
        df=read_pdf(f,pages='1')
        # get the file name info
        fname_track=df[0]['Box Score'][0].replace(" ","")
        date_text=df[0]['Box Score'][2]
        pattern=r'(\d*)/(\d*)/(\d*)'
        fname_year=re.search(pattern,date_text).group(3)
        fname_day=re.search(pattern,date_text).group(2).zfill(2)
        fname_month=re.search(pattern,date_text).group(1).zfill(2)
        # getting the data cleaned up
        print(f)
        # dataframe cleanup steps:
        # 1. removed extra header information
        df2=df[0].iloc[5:]
        # 2. handling merged columns 
        df2['Box Score']=df2['Box Score'].str.split()
        df2['Unnamed: 7']=df2['Unnamed: 7'].str.split()
        df2['Unnamed: 9']=df2['Unnamed: 9'].str.split()

        #split merged columns into the respected columns, this information can be gathered from the PDF.
        df3=pd.DataFrame(df2['Box Score'].tolist(), columns=['PassDiff','GreenPass','GreenPassed','QualityPasses','PercentQualityPasses','NumFastestLaps'])
        df4=pd.DataFrame(df2['Unnamed: 6'].tolist(), columns=['LapsinTop15','PercentLapsTop15'])
        # resetting the index to prep for the concat
        df2=df2[['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
            'Unnamed: 5', 'Unnamed: 7', 'Unnamed: 8',
            'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11']]
        df2=df2.reset_index(drop=True)
        # combine the split up columns with the properly parsed columns
        df_final=pd.concat([df2,df3,df4],axis=1)
        df_final.columns=HEADERS


        # output the data
        df_final.to_csv(r'.\SIADS591_592_Milestone1\Milestone_I\Milestone_I\data\loop_TobyChristie\csv\2022\{}.csv'.format(fname_year+fname_month+fname_day+'_'+fname_track),index=False)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9ef4eb23-f38f-47e8-bfb0-51d8616b0dee' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>